# Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

2021-08-27 17:30:50.097168: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-27 17:30:50.097247: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Dataset

In [3]:
dataset = pd.read_csv('../data/processed_data/active_users_df_10PlusRatings_partial.csv')
anime_df = pd.read_csv('../data/raw_data/anime.csv')
dataset = dataset.sort_values(["user_id", "anime_id"], ascending=(True, True))

In [5]:
dataset.anime_id.nunique()

12106

In [6]:
dataset.user_id.nunique()

4035

In [7]:
anime_id_to_new_id = dict()
id = 1

In [8]:
for index, row in dataset.iterrows():
    if anime_id_to_new_id.get(row['anime_id']) is None:
        anime_id_to_new_id[row['anime_id']] = id
        dataset.at[index, 'anime_id'] = id
        id += 1
    else:
        dataset.at[index, 'anime_id'] = anime_id_to_new_id.get(row['anime_id'])


In [31]:
# Need to map user ID to [1, user_num]
user_id_to_new_id = dict()
id = 1
for index, row in dataset.iterrows():
    if user_id_to_new_id.get(row['user_id']) is None:
        user_id_to_new_id[row['user_id']] = id
        dataset.at[index, 'user_id'] = id
        id += 1
    else:
        dataset.at[index, 'user_id'] = user_id_to_new_id.get(row['user_id'])


In [33]:
dataset.user_id

3            1
4            1
0            1
9            1
14           1
          ... 
822308    4035
822416    4035
822372    4035
822305    4035
822356    4035
Name: user_id, Length: 822418, dtype: int64

In [34]:
num_users = len(dataset.user_id.unique())
num_animes = len(dataset.anime_id.unique())
train, test = train_test_split(dataset, test_size=0.2)

print('Number of movies', num_animes)
print('Number of users', num_users)

Number of movies 12106
Number of users 4035


# Order dataset by user_id and anime_id

In [35]:
print('train shape: ', train.shape)
print('test shape: ', test.shape)

train shape:  (657934, 3)
test shape:  (164484, 3)


In [36]:
#train = train.reset_index()

In [37]:
# 151519


#train.anime_id.nunique()
#num_animes

In [38]:
#num_animes

# GMF


## Matrix factorisation

In [39]:
import pydot
import graphviz
%matplotlib inline

In [40]:
from keras.models import Model, Sequential
from keras.layers import Embedding, Flatten, Input, merge, concatenate
from tensorflow.keras.utils import model_to_dot
#from keras.utils.visualize_util import model_to_dot
from IPython.display import SVG

In [41]:
# Let's use a higher latent dimension.
latent_dim = 10

anime_input = Input(shape=[1],name='anime-input')
anime_embedding = Embedding(num_animes + 1, latent_dim, name='anime-embedding')(anime_input)
anime_vec = Flatten(name='anime-flatten')(anime_embedding)

user_input = Input(shape=[1],name='user-input')
user_embedding = Embedding(num_users + 1, latent_dim, name='user-embedding')(user_input)
user_vec = Flatten(name='user-flatten')(user_embedding)
from keras.layers import dot
prod = dot([anime_vec, user_vec], axes=1, normalize=False)

model = Model([user_input, anime_input], prod)
model.compile('adam', 'mean_squared_error'
              #,metrics=['mse']
             )

In [42]:
print('user_input',user_input.shape)
print('anime_input',anime_input.shape)

user_input (None, 1)
anime_input (None, 1)


In [43]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anime-input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
user-input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
anime-embedding (Embedding)     (None, 1, 10)        121070      anime-input[0][0]                
__________________________________________________________________________________________________
user-embedding (Embedding)      (None, 1, 10)        40360       user-input[0][0]                 
____________________________________________________________________________________________

In [44]:
dataset.rating = dataset.rating.astype(float)

In [45]:
dataset.dtypes

user_id       int64
anime_id      int64
rating      float64
dtype: object

In [46]:
train.user_id.shape

(657934,)

In [47]:
train.anime_id.shape

(657934,)

In [48]:
train.rating.shape

(657934,)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

history = model.fit([train.user_id, train.anime_id], train.rating, epochs=4)

Epoch 1/4
20561/20561 [==============================] - 79s 4ms/step - loss: 20.8802
Epoch 2/4
20561/20561 [==============================] - 73s 4ms/step - loss: 3.0764
Epoch 3/4
20561/20561 [==============================] - 75s 4ms/step - loss: 2.2927
Epoch 4/4
20561/20561 [==============================] - 63s 3ms/step - loss: 2.0560


In [ ]:
pd.Series(history.history['loss']).plot(logy=True)
plt.xlabel("Epoch")
plt.ylabel("Train Error")
plt.show()

y_hat = np.round(model.predict([test.user_id, test.anime_id]), decimals=2)
y_true = test.rating
mean_absolute_error(y_true, y_hat)